In [1]:
import cfg

running price streamrunning transaction stream

running account update polling


In [2]:
cfg.account

In [3]:
print(cfg.account.trades)

[<v20.trade.TradeSummary object at 0x7feaec29a1f0>, <v20.trade.TradeSummary object at 0x7feaec29a220>, <v20.trade.TradeSummary object at 0x7feaec29a250>, <v20.trade.TradeSummary object at 0x7feaec29a1c0>, <v20.trade.TradeSummary object at 0x7feaec29a760>]


In [4]:
type(cfg.account.trades)

list

In [5]:
for t in cfg.account.trades:
    print(t)
    break

id: '165755'
instrument: EUR_USD
price: 1.13854
openTime: '2021-12-31T17:58:03.092430173Z'
state: OPEN
initialUnits: 385.0
initialMarginRequired: 8.7662
currentUnits: 385.0
realizedPL: 0.0
unrealizedPL: -0.2964
marginUsed: 8.7615
financing: 0.0
takeProfitOrderID: '165756'
stopLossOrderID: '165757'
trailingStopLossOrderID: '165758'


In [9]:
def check_breakeven_for_instrument(instrument: str, trades: list):
    all_breakeven = []
    for t in trades:
        if t.instrument == instrument:
            for o in cfg.account.orders:
                if o.id == t.stopLossOrderID:
                    all_breakeven.append(
                    (t.currentUnits > 0 ) and (o.price >=t.price)
                    or
                    (t.currentUnits <0 and o.price<=t.price))
    return all(all_breakeven)

In [10]:
check_breakeven_for_instrument('AUD_USD', cfg.account.trades)

False

In [11]:
for o in cfg.account.orders:
    print(o)
    break

id: '165645'
createTime: '2021-12-31T16:22:02.456740543Z'
state: PENDING
type: TAKE_PROFIT
tradeID: '165644'
price: 0.73642
timeInForce: GTC
triggerCondition: DEFAULT


In [ ]:
for t in cfg.account.trades:
    check_breakeven(t)
    break

In [4]:
cfg.account.balance

1608.9

In [7]:
cfg.instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.1312,
 'ask': 1.1317,
 'spread': 0.0005}

In [ ]:
cfg.account.marginAvailable